In [1]:
import cv2
from matplotlib import pyplot as plt
import sys
import random
import numpy as np
from scipy.ndimage import gaussian_filter

In [2]:
img1 = cv2.imread('HW2/uttower_left.JPG',0)
img2 = cv2.imread('HW2/uttower_right.JPG',0)

In [3]:
#computing sift key-points and descriptors
sift = cv2.xfeatures2d.SIFT_create()
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)

In [4]:
dist_arr = []
l1 = len(des1)
l2 = len(des2)
#calculating the distance between descriptors
for i in range(l1):
    temp_arr = []
    for j in range(l2):
        temp = np.linalg.norm(des1[i]-des2[j])
        temp_arr.append((temp,i,j))
    top = sorted(temp_arr, key=lambda x: x[0])[0:2]
    dist_arr.append([top[0],top[1]])

In [5]:
#taking top 100 descriptors with least distance
top_matches = sorted(dist_arr, key=lambda x: x[0][0])[0:100]

In [6]:
#among top 100 we are keeping good descriptors
good_matches = []
for i in top_matches:
    if i[0][0]/i[1][0] < 0.5:
        good_matches.append(i[0])

In [7]:
lkp = []
rkp = []
for _, i, j in good_matches:
    lkp.append(kp1[int(i)])
    rkp.append(kp2[int(j)])
L = np.array([])
R = np.array([])
L = cv2.drawKeypoints(img1, lkp, L, color=(255,0,0))
R = cv2.drawKeypoints(img2, rkp, R, color=(255,0,0))
plt.imsave('leftKeyPoints.jpg',L)
plt.imsave('rightKeyPoints.jpg',R)

In [8]:
left = cv2.imread('HW2/uttower_left.JPG')
right = cv2.imread('HW2/uttower_right.JPG')
hA, wA = left.shape[:2]
hB, wB = right.shape[:2]

vis = np.zeros((max(hA, hB), wA + wB, 3), dtype="uint8")
vis[0:hA, 0:wA] = left
vis[0:hB, wA:] = right

for i in range(len(lkp)):
    ptA = (int(lkp[i].pt[0]), int(lkp[i].pt[1]))
    ptB = (int(rkp[i].pt[0]) + wA, int(rkp[i].pt[1]))
    cv2.line(vis, ptA, ptB, (255, 0, 0), 2)
cv2.imwrite("Matchings.jpg", vis)

True

In [12]:
#function to calculate homography
def get_homography(p):
    A = []
    for i in range(4):
        u1,v1 = kp1[p[i][1]].pt
        u,v = kp2[p[i][2]].pt
        A.append([-u,-v,-1,0,0,0,u*u1,v*u1,u1])
        A.append([0,0,0,-u,-v,-1,u*v1,v*v1,v1])
    u, s, v = np.linalg.svd(A)
    h = v[-1,:].reshape(3,3)
    h = h/v[-1,-1]
    return h

In [13]:
def get_inliers(h,good_matches,delta):
    inc = 0
    l = len(good_matches)
    for q in range(l):
        u1,v1 = kp1[good_matches[q][1]].pt
        u,v = kp2[good_matches[q][2]].pt
        x = np.array([[u],[v],[1]])
        y = np.array([[u1],[v1],[1]])
        x1 = h.dot(x)
        dist = np.linalg.norm(y-x1)
        if(dist<=delta):
            inc+=1
    return inc

In [14]:
#RANSAC
N = 10
H = []
temp_inc = 0
l = len(good_matches)
l1 = len(lkp)
for k in range(N):
    il = []
    while(len(set(il))<4):
        il.append(random.randint(0, l1-1))
        il = list(set(il))
    h = get_homography([good_matches[il[0]],good_matches[il[1]],good_matches[il[2]],good_matches[il[3]]])
    inc = get_inliers(h, good_matches ,150)
    if(temp_inc<=inc):
        temp_inc = inc
        H = h

In [15]:
left = cv2.imread('HW2/uttower_left.JPG')
right = cv2.imread('HW2/uttower_right.JPG')
warped_img = cv2.warpPerspective(right,H,(left.shape[1] + right.shape[1], left.shape[0]))
warped_img[0:left.shape[0], 0:left.shape[1]] = left
cv2.imwrite('stitched_panorama.jpg',warped_img)

True

In [16]:
ws = 3
cfirst = left[0:left.shape[0],0:left.shape[1]-ws]
first = left[0:left.shape[0],left.shape[1]-ws:left.shape[1]]
temp_img = warped_img[0:left.shape[0],left.shape[1]:warped_img.shape[1]]
csecond = temp_img[0:left.shape[0],ws:temp_img.shape[1]]
second = temp_img[0:left.shape+[0],0:ws]
bl1 = cv2.addWeighted(first, 0.55,second, 0.55, 0)
bl2 = cv2.addWeighted(csecond, 0.595, csecond, 0.595, 0)
p1 = np.concatenate((cfirst, bl1), axis=1)
final_img = np.concatenate((p1, bl2), axis=1)
cv2.imwrite('final_stitched_img.jpg',final_img)

True